In [1]:
from cobra.io import read_sbml_model
model = read_sbml_model('iYO844.xml')

In [2]:
model

Name,iYO844
Memory address,0x02957007a940
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


In [3]:
max_growth=model.optimize().objective_value

In [4]:
model.objective=model.reactions.EX_ribflv_e
model.reactions.BIOMASS_BS_10.lower_bound=0.2*max_growth
solution_ribflv=model.optimize()
print(solution_ribflv.objective_value,solution_ribflv.fluxes.BIOMASS_BS_10)

0.3030802735713299 0.02359327786447964


In [5]:
%%time
copy_of_model = model.copy()

Wall time: 350 ms


In [6]:
%%time
for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

Wall time: 4min 14s


In [7]:
%%time
with model:
    for gene in model.genes:
        gene.knock_out()

Wall time: 149 ms


In [8]:
with model:
    model.reactions.bounds = 0, 0
    print("BGLA1's bounds inside the with statement")
    print(model.reactions.BGLA1.lower_bound, model.reactions.BGLA1.bounds)
    print('Mutant growth rate: ', model.optimize().objective_value)
print("BGLA1's bounds outside the with statement")
print(model.reactions.BGLA1.bounds)

BGLA1's bounds inside the with statement
0.0 (0.0, 999999.0)
Mutant growth rate:  0.3030802735713299
BGLA1's bounds outside the with statement
(0.0, 999999.0)


In [9]:
%%time
solution = model. optimize()
solution.fluxes['ATPM']

Wall time: 12.6 ms


9.0

In [10]:
%%time
model.slim_optimize()
model.reactions.ATPM.flux

Wall time: 0 ns


9.0

In [11]:
model.medium

{'EX_fe3_e': 999999.0,
 'EX_glc__D_e': 1.7,
 'EX_ca2_e': 999999.0,
 'EX_so4_e': 5.0,
 'EX_co2_e': 999999.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_mg2_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_nh4_e': 5.0,
 'EX_o2_e': 999999.0,
 'EX_pi_e': 5.0}

In [12]:
solution = model.optimize()

In [13]:
solution.fluxes

EX_etha_e    0.000000
EX_drib_e    0.000000
EX_csn_e     0.000000
EX_5mtr_e    0.000000
EX_dtmp_e    0.000000
               ...   
GND          0.000000
PGK         -1.815763
PGLYCP       0.000000
PGM         -1.489707
PHCYT_BS     0.001812
Name: fluxes, Length: 1250, dtype: float64

In [14]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] =0
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_ribflv_e'])
    
medium = model.medium
with model:
    medium['EX_glc__D_e'] =0
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_BS_10'])

-0.23735877977639705
0.02359327786447964


C:\Users\13512\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\13512\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [15]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.7
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_ribflv_e'])

0.30308027357133016


In [30]:
with model:
    # calculating max yield of riboflavin
    substrate_flux = model.reactions.EX_ribflv_e.flux
    model.objective = model.reactions.EX_ribflv_e 
    max_ribflv_e_production = model.optimize().objective_value
    print("The flux of ribflv_e production: ",max_ribflv_e_production , "[mmol gDW^-1 h^-1]")
with model:
    # calculating max yield of biomass
    substrate_flux = model.reactions.EX_glc__D_e.flux
    model.objective = model.reactions.BIOMASS_BS_10
    max_biomass_production = model.optimize().objective_value
    print("The flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass: ", molar_yield_biomass, "[mmol-biomass / mmol-glc]")
    # calculating molar yield
    molar_yield_ribflv_e = max_ribflv_e_production / (-1. * substrate_flux)
    print("The molar yield of ribflv_e: ", molar_yield_ribflv_e, "[mmol-ribflv_e / mmol-glc]")

The flux of ribflv_e production:  0.3030802735713336 [mmol gDW^-1 h^-1]
The flux of biomass production:  0.11796638932239904 [mmol gDW^-1 h^-1]
The molar yield of biomass:  0.06939199371905826 [mmol-biomass / mmol-glc]
The molar yield of ribflv_e:  0.17828251386549035 [mmol-ribflv_e / mmol-glc]


In [18]:
from cobra.medium import minimal_medium

In [20]:
model

Name,iYO844
Memory address,0x02957007a940
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*EX_ribflv_e - 1.0*EX_ribflv_e_reverse_d585d
Compartments,"cytosol, extracellular space"


In [22]:
max_growth=model.slim_optimize()

In [27]:
minimal_medium(model)

EX_fe3_e       0.000081
EX_glc__D_e    1.642280
EX_ca2_e       0.000076
EX_so4_e       0.004018
EX_k_e         0.016664
EX_mg2_e       0.002400
EX_nh4_e       0.588993
EX_o2_e        2.548431
EX_pi_e        0.036211
dtype: float64

In [29]:
# Glucose content with minimal medium
medium=model.medium
with model:
    medium['EX_glc__D_e'] = 1.64
    model.medium = medium
    solution = model.optimize()
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.64
    model.medium = medium
    solution = model.optimize()

with model:
    # calculating max yield of riboflavin
    substrate_flux = model.reactions.EX_glc__D_e.flux
    model.objective = model.reactions.EX_ribflv_e 
    max_ribflv_e_production = model.optimize().objective_value
    print("The flux of ribflv_e production: ",max_ribflv_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_ribflv_e = max_ribflv_e_production / (-1. * substrate_flux)
    print("The molar yield of ribflv_e: ", molar_yield_ribflv_e, "[mmol-ribflv_e / mmol-glc]")   
with model:
    # calculating max yield of riboflavin
    substrate_flux = model.reactions.EX_glc__D_e.flux
    model.objective = model.reactions.BIOMASS_BS_10
    max_biomass_production = model.optimize().objective_value
    print("The flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass: ", molar_yield_biomass, "[mmol-biomass / mmol-glc]")

The flux of ribflv_e production:  0.3030802735713336 [mmol gDW^-1 h^-1]
The molar yield of ribflv_e:  0.18480504486056928 [mmol-ribflv_e / mmol-glc]
The flux of biomass production:  0.11796638932239904 [mmol gDW^-1 h^-1]
The molar yield of biomass:  0.06939199371905826 [mmol-biomass / mmol-glc]


In [32]:
# Anaerobic condition 
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.64
    medium['EX_o2_e'] = 0
    model.medium = medium
    solution = model.optimize()

medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.64
    medium['EX_o2_e'] = 0
    model.medium = medium
    solution = model.optimize()
    
with model:
    # calculating max yield of riboflavin
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_o2_e.flux
    model.objective = model.reactions.EX_ribflv_e 
    max_ribflv_e_production = model.optimize().objective_value
    print("The flux of ribflv_e production: ",max_ribflv_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_ribflv_e = max_ribflv_e_production / (-1. * substrate_flux)
    print("The molar yield of ribflv_e: ", molar_yield_ribflv_e, "[mmol-ribflv_e / mmol-sub]")
with model:
    # calculating max yield of riboflavin
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_o2_e.flux
    model.objective = model.reactions.BIOMASS_BS_10
    max_biomass_production = model.optimize().objective_value
    print("The flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass: ", molar_yield_biomass, "[mmol-biomass / mmol-sub]")

The flux of ribflv_e production:  0.3030802735713295 [mmol gDW^-1 h^-1]
The molar yield of ribflv_e:  0.08378720892328533 [mmol-ribflv_e / mmol-sub]
The flux of biomass production:  0.11796638932239743 [mmol gDW^-1 h^-1]
The molar yield of biomass:  0.01917561667316124 [mmol-biomass / mmol-sub]


C:\Users\13512\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
C:\Users\13512\anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


In [33]:
# Aerobic condition
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.64
    medium['EX_o2_e'] = 4
    model.medium = medium
    solution = model.optimize()
   
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 1.64
    medium['EX_o2_e'] = 4
    model.medium = medium
    solution = model.optimize()
with model:
    # calculating max yield of riboflavin
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_o2_e.flux
    model.objective = model.reactions.EX_ribflv_e 
    max_ribflv_e_production = model.optimize().objective_value
    print("The flux of ribflv_e production: ",max_ribflv_e_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_ribflv_e = max_ribflv_e_production / (-1. * substrate_flux)
    print("The molar yield of ribflv_e: ", molar_yield_ribflv_e, "[mmol-ribflv_e / mmol-sub]")
with model:
    # calculating max yield of riboflavin
    substrate_flux = model.reactions.EX_glc__D_e.flux + model.reactions.EX_o2_e.flux
    model.objective = model.reactions.BIOMASS_BS_10
    max_biomass_production = model.optimize().objective_value
    print("The flux of biomass production: ",max_biomass_production , "[mmol gDW^-1 h^-1]")
    # calculating molar yield
    molar_yield_biomass = max_biomass_production / (-1. * substrate_flux)
    print("The molar yield of biomass: ", molar_yield_biomass, "[mmol-biomass / mmol-sub]")

The flux of ribflv_e production:  0.30308027357133355 [mmol gDW^-1 h^-1]
The molar yield of ribflv_e:  0.053737637158037865 [mmol-ribflv_e / mmol-sub]
The flux of biomass production:  0.11796638932240001 [mmol gDW^-1 h^-1]
The molar yield of biomass:  0.019175616673161686 [mmol-biomass / mmol-sub]
